<a href="https://colab.research.google.com/github/Daniel022de/DataEnginner_SoulCode/blob/main/ETL/Credit%20Card%20Transactions/etl_CreditCardTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📗 **ETL Credit Card Transactions**

Esse Notebook é referente a implementação de um processo ETL aplicado a um dataset público disponível no [kaggle.com](https://www.kaggle.com/datasets/thedevastator/analyzing-credit-card-spending-habits-in-india?resource=download) e carregado no meu Google Cloud Storage privado.



! **Você pode encontrar esse notebook no meu repositório** [GitHub](https://github.com/Daniel022de/Bootcamp_SoulCode_EngenhariaDados)

! **Você pode entrar em contato comigo através do meu email** ddololiveira.pessoal@gmail.com **e** [Linkedin](https://www.linkedin.com/in/daniel-oliveira-503b0323b/).

! **Toda dúvida,recomendações e feedbacks serão bem-vindas.**



#▶ 1 Instalação

In [ ]:
#Instalando gcsfs para conexao ao Google Cloud Storage
!pip install gcsfs

In [ ]:
#Instalação PySpark
!pip install pyspark

In [ ]:
#Instalação pandera
!pip install pandera

#▶ 2 Importando bibliotecas

In [116]:
import pandas as pd
import os
import pyspark.sql.functions as F
import pandera as pa
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from google.cloud import storage
from google.colab import drive
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

#▶ 3  Conexão ao Google Drive

**Conexão utilizada para ter acesso a chave de acesso ao Google Cloud Storage**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#▶ 4 Conexão Storage GCP

**Conexão feita com a finalidade de extração do DataSet.**

In [118]:
ServiceAccount = '/content/drive/MyDrive/ACESSO A GCP/bc26-aulas-ae110e89e3a6.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ServiceAccount

#▶ 5 Extração do DataSet

In [119]:
storage.Client().get_bucket('ed7-datasets').blob('Credit card transactions - India - Simple.csv')
path='gs://ed7-datasets/brutos/Credit card transactions - India - Simple.csv'

#▶ 6 Carregando - DataFrame - Pandas

In [120]:
df = pd.read_csv(path)

#▶ 7  Pré-análise do DataFrame

**Sobre o DataFrame**:

Este DataFrame contém informações sobre transações de cartão de crédito feitas na Índia, **oferecendo uma visão abrangente dos hábitos de consumo dos indianos em todo o país**.

**Colunas:**

* City:  A cidade em que a transação ocorreu (String)

* Date:  A data da transação. (Date)

* Card Type: o tipo de cartão de crédito usado para a transação. (String)

* Exp Type: O tipo de despesa associada à transação. (String)

* Gender: o sexo do titular do cartão. (String)

* Amount: o valor da transação. (Number)


Fonte das informações [aqui.](https://www.kaggle.com/datasets/thedevastator/analyzing-credit-card-spending-habits-in-india?resource=download)

**LIMPEZAS | TRANSFORMAÇÕES | TRATAMENTOS**

1. Dropar coluna 'index'
2. Traduzir Dados
 

In [121]:
#Visualização geral do DataFrame
df

,index,City,Date,Card Type,Exp Type,Gender,Amount
0,0,"Delhi, India",29-Oct-14,Gold,Bills,F,82475
1,1,"Greater Mumbai, India",22-Aug-14,Platinum,Bills,F,32555
2,2,"Bengaluru, India",27-Aug-14,Silver,Bills,F,101738
3,3,"Greater Mumbai, India",12-Apr-14,Signature,Bills,F,123424
4,4,"Bengaluru, India",5-May-15,Gold,Bills,F,171574
...,...,...,...,...,...,...,...
26047,26047,"Kolkata, India",22-Jun-14,Silver,Travel,F,128191
26048,26048,"Pune, India",3-Aug-14,Signature,Travel,M,246316
26049,26049,"Hyderabad, India",16-Jan-15,Silver,Travel,M,265019
26050,26050,"Kanpur, India",14-Sep-14,Silver,Travel,M,88174


##◼ 7.1 Verificando Dados

In [122]:
#Verificando dados nulos

df.isnull().sum()

index        0
City         0
Date         0
Card Type    0
Exp Type     0
Gender       0
Amount       0
dtype: int64

In [123]:
#Verificando dados duplicados

df.duplicated().sum()

0

## ◼  7.2 Tipo dos dados

**LIMPEZAS | TRANSFORMAÇÕES | TRATAMENTOS**

3. Converter coluna de datas para Date

In [124]:
df.dtypes

index         int64
City         object
Date         object
Card Type    object
Exp Type     object
Gender       object
Amount        int64
dtype: object

##◼ 7.3  Analisando Colunas

In [125]:
df.groupby(['Date'],dropna=False).size().sort_values()

Date
11-Apr-15    23
30-Jul-14    24
28-Feb-14    25
9-May-14     26
29-Aug-14    27
             ..
12-Jan-15    60
15-Nov-14    61
11-Aug-14    61
21-Dec-13    61
20-Sep-14    65
Length: 600, dtype: int64

#▶ 8 Tratamento | Limpeza Pandas

## ◼  8.1  backup

In [126]:
df_backup = df.copy()

## ◼  8.2 Dropando coluna

**A coluna index é desnecessário e redundante com os labels do dataframe.**

In [127]:
df.drop(['index'],axis=1,inplace=True)

##◼  8.3  Traduzindo dados

In [128]:
#Traduzindo colunas

df.rename(
    columns = {
        'City':'cidade_transacao',
        'Date':'data_transacao',
        'Card Type':'tipo_cartao',
        'Exp Type':'tipo_despesa',
        'Gender':'sexo_titular',
        'Amount':'valor_transacao'

    },inplace=True
)

In [129]:
#Traduzindo dados das colunas

df['tipo_despesa'].replace(['Travel','Grocery','Entertainment','Bills','Fuel','Food'],['Viagem','Supermercado','Entretenimento','Contas','Combustível','Comida'],inplace=True)

In [130]:
df['sexo_titular'].replace(['F','M'],['Feminino','Masculino'],inplace=True)

#▶ 9 Tratamento | Limpeza Pyspark

##◼ 9.1 SparkSession - PySpark

In [131]:
spark =  (
    SparkSession.builder
                .master('local')
                .appName('gcsfs')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

## ◼ 9.2  DataFrame Pandas > DataFrame Pyspark

In [132]:
df_pyspark = spark.createDataFrame(df)

In [133]:
df_pyspark.show(truncate=False)

+---------------------+--------------+-----------+------------+------------+---------------+
|cidade_transacao     |data_transacao|tipo_cartao|tipo_despesa|sexo_titular|valor_transacao|
+---------------------+--------------+-----------+------------+------------+---------------+
|Delhi, India         |29-Oct-14     |Gold       |Contas      |Feminino    |82475          |
|Greater Mumbai, India|22-Aug-14     |Platinum   |Contas      |Feminino    |32555          |
|Bengaluru, India     |27-Aug-14     |Silver     |Contas      |Feminino    |101738         |
|Greater Mumbai, India|12-Apr-14     |Signature  |Contas      |Feminino    |123424         |
|Bengaluru, India     |5-May-15      |Gold       |Contas      |Feminino    |171574         |
|Delhi, India         |8-Sep-14      |Silver     |Contas      |Feminino    |100036         |
|Delhi, India         |24-Feb-15     |Gold       |Contas      |Feminino    |143250         |
|Greater Mumbai, India|26-Jun-14     |Platinum   |Contas      |Feminin

## ◼ 9.3  Verificando tamanho dos dados

In [134]:
#Total de linhas

df_pyspark.count()

26052

In [135]:
#Linhas distintas

df_pyspark.distinct().count()

26052

##◼ 9.4  Coluna cidade_trasacao

**Retirar India, pois todas as transições ocorreram neste País. Portanto é desnecessário manter esse dado.**

In [136]:
df_pyspark = df_pyspark.withColumn('cidade_transacao',F.regexp_replace('cidade_transacao',', India',''))

##◼  9.5  Coluna data_trasacao

In [137]:
# Tratando os meses

df_pyspark = (df_pyspark.withColumn('data_transacao',F.regexp_replace('data_transacao','Jan','01'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Feb','02'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Mar','03'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Apr','04'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','May','05'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Jun','06'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Jul','07'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Aug','08'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Sep','09'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Oct','10'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Nov','11'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','Dec','12')) 
)
                        

**O levantamento dos dados foi feito entre o ano de 2013 e 2015.**

In [138]:
# Tratando os anos

df_pyspark = (df_pyspark.withColumn('data_transacao',F.regexp_replace('data_transacao','-13','-2013'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','-14','-2014'))
                        .withColumn('data_transacao',F.regexp_replace('data_transacao','-15','-2015')))

**Para converter a coluna 'data_trasacao' para DateType precisei antes realizar um tratamento nos meses e anos. Tive problema para tratar os dias no intervalo [1,9], mas utilizando a função func abaixo esse tratamento foi dispensado.**

In [139]:
# Esta função converte os dados String da coluna para DateType

func =  udf (lambda x: datetime.strptime(x, '%d-%m-%Y'), DateType())

df_pyspark = df_pyspark.withColumn('data_transacao', func(col('data_transacao')))


#▶ 10 Validação - Pandera

In [140]:
#Convertendo pra DataFrame Pandas
df_pandas_tratado = df_pyspark.toPandas()

In [141]:
df_pandas_tratado.head(1)

,cidade_transacao,data_transacao,tipo_cartao,tipo_despesa,sexo_titular,valor_transacao
0,Delhi,2014-10-29,Gold,Contas,Feminino,82475


In [142]:
#Validação
schema = pa.DataFrameSchema(
    columns = {
        'cidade_transacao':pa.Column(pa.String),
        'data_transacao':pa.Column(pa.Date),
        'tipo_cartao':pa.Column(pa.String),
        'tipo_despesa':pa.Column(pa.String),
        'sexo_titular':pa.Column(pa.String),
        'valor_transacao':pa.Column(pa.Int)
        
}
)

In [143]:
schema.validate(df_pandas_tratado)

,cidade_transacao,data_transacao,tipo_cartao,tipo_despesa,sexo_titular,valor_transacao
0,Delhi,2014-10-29,Gold,Contas,Feminino,82475
1,Greater Mumbai,2014-08-22,Platinum,Contas,Feminino,32555
2,Bengaluru,2014-08-27,Silver,Contas,Feminino,101738
3,Greater Mumbai,2014-04-12,Signature,Contas,Feminino,123424
4,Bengaluru,2015-05-05,Gold,Contas,Feminino,171574
...,...,...,...,...,...,...
26047,Kolkata,2014-06-22,Silver,Viagem,Feminino,128191
26048,Pune,2014-08-03,Signature,Viagem,Masculino,246316
26049,Hyderabad,2015-01-16,Silver,Viagem,Masculino,265019
26050,Kanpur,2014-09-14,Silver,Viagem,Masculino,88174


#▶ 11 Carregando no bucket

In [144]:
df_pandas_tratado.to_csv('gs://ed7-datasets/tratados/CreditCardTransactions_tratado.csv',storage_options={'token':'/content/drive/MyDrive/ACESSO A GCP/bc26-aulas-ae110e89e3a6.json'})